In [ ]:
import os,tqdm,cv2,sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator

#取消像素限制
Image.MAX_IMAGE_PIXELS = None

#### Remove the  taproot

In [1]:

import os,tqdm,cv2,sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator

#取消像素限制
Image.MAX_IMAGE_PIXELS = None
#滤波
def blus(image_array):
    img_blur=cv2.blur(image_array,(8,8))
    return img_blur
#阈值
def threshold(image_array, lower, upper):
    result = np.where(image_array>lower,255,0)
    return result
# Scharr算子实现梯度计算
def Scharr_demo(image_array):
    # x 方向梯度
    image_grad_x = cv2.Scharr(image_array, cv2.CV_32F, 1, 0)
    # 分别求绝对值并转化为8位的图像上，这样做方便显示
    image_gradx = cv2.convertScaleAbs(image_grad_x) 
    return image_gradx
#形态学操作
def morphologyEx_demo(image_array):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,40))#3,40
    result  = cv2.morphologyEx(image_array,cv2.MORPH_OPEN,kernel,iterations=4)
    return result
#图片相减
def math_demo(image1_array,image2_array,add_or_sub="add"):
    if add_or_sub=="add":
        image = image1_array + image2_array
    if add_or_sub=="sub":
        image = image1_array - image2_array
    return image

# 分割主根操作(基于egnet)
#大图分割结果
image1_file = r"D:\31890\Desktop\EGNet_ResNet50"
save_file   = r"D:\31890\Desktop\EGNet_ResNet50_no"
if not os.path.exists(save_file):
    os.makedirs(save_file)
for name in tqdm.tqdm(os.listdir(image1_file)):
    image1_path  = os.path.join(image1_file,name)
    # image2_path  = os.path.join(image2_file,name)
    save_path = os.path.join(save_file,name)
    image = cv2.imread(image1_path,cv2.IMREAD_GRAYSCALE)
    binary = threshold(image, 35, 255)
    open_image = morphologyEx_demo(image)
    cv2.imwrite(os.path.join(save_file,'1_'+name),open_image)
    y_grady = Scharr_demo(open_image)
    cv2.imwrite(os.path.join(save_file,'2_'+name),y_grady)
    image = math_demo(open_image,y_grady,'add')
    cv2.imwrite(os.path.join(save_file,'3_'+name),image)
    blus_image = blus(image)
    cv2.imwrite(os.path.join(save_file,'4_'+name),blus_image)
    result = threshold(blus_image, 35, 255)
    cv2.imwrite(os.path.join(save_file,'5_'+name),result)
    result = math_demo(binary,result,'sub')
    cv2.imwrite(os.path.join(save_file,'6_'+name),result)
    cv2.imwrite(save_path,result)

100%|██████████| 41/41 [00:12<00:00,  3.21it/s]


#### SLIC

In [ ]:
import cv2,os,tqdm
import numpy as np
from PIL import Image

def shuailao_slic(image_array,mask_array):
    #初始化slic项，超像素平均尺寸20（默认为10），平滑因子20
    slic = cv2.ximgproc.createSuperpixelSLIC(image_array,region_size=20,ruler = 20)
    slic.iterate(50)                                    #迭代次数，越大效果越好

    label_slic = (slic.getLabels() +1).astype(np.uint8)                 #获取超像素标签矩阵
    label_slic = cv2.bitwise_and(label_slic, label_slic, mask=mask_array.astype(np.uint8)).astype(np.uint8)

    class_num = np.unique(label_slic,return_index=False,return_inverse=False)
    for i in range(len(np.unique(label_slic,return_index=False,return_inverse=False))):
        label_slic[np.where(label_slic==class_num[i])] = i  #重新排列分类矩阵

    image_gray = cv2.cvtColor(image_array,cv2.COLOR_BGR2GRAY)
    image_gray = cv2.bitwise_and(image_gray, image_gray, mask=mask_array).astype(np.uint8)

    image_copy = image_gray.copy()

    # a1,b1 = np.unique(image_gray,return_counts=True)
    # print(a1,b1)

    for i in range(len(class_num)):
        a,b = np.unique(image_gray[label_slic==i],return_counts=True)
        # print(a,b)
        # print(a[np.argmax(b)])
        if  a[np.argmax(b)]==0:
            image_copy[np.where(label_slic==i)] = a[np.argmax(b)]
            # a2,b2 = np.unique(image_copy[label_slic==i],return_counts=True)
            # print(a2,b2)
        elif a[np.argmax(b)]==141:
            image_copy[np.where(label_slic==i)] = 1
        elif a[np.argmax(b)]==76:
            image_copy[np.where(label_slic==i)] = 2
        else:
            image_copy[np.where(label_slic==i)] = 3
            

    image_copy = Image.fromarray(image_copy)
    image_copy.convert('L')
    #调色板
    palette = [0, 0, 0,0, 255, 0, 255, 0, 0,255,255,255]
    # palette = [0, 0, 0, 128, 0, 0,0, 128, 0, 128, 128, 0,
    #         0, 0, 128,]
            # 128, 0, 128, 0, 128, 128, 128, 128, 128,
            # 64, 0, 0
    #着色
    image_copy.putpalette(palette)

    return image_copy

def main(i,m,o):
    if not os.path.exists(o):
        os.makedirs(o)
    for name in tqdm.tqdm(os.listdir(i)):
        name  = name.split('.')[0]
        image_array = cv2.imread(os.path.join(i,name+'.png'))
        mask_array  = cv2.imread(os.path.join(m,name+'.png'),cv2.IMREAD_GRAYSCALE)
        image_rebuild = shuailao_slic(image_array,mask_array)
        image_rebuild.save(os.path.join(o,name+'.png'))

i = r"D:\31890\Desktop\tranformer\senescence\shuailao"
m = r"D:\31890\Desktop\tranformer\senescence\mask_heidi"
o = r"D:\31890\Desktop\tranformer\senescence\shuailao_re2"
main(i,m,o)
